## <font color=blue>Create un fichero environ.py con tus datos reales</font>

In [ ]:
# --- EJEMPLO environ.py -----

# -- Divisa --------------
os.environ['BADGE']='USD' # 'EUR'

# -- API Coinmarketcap ---------
os.environ['X-CMC_PRO_API_KEY'] = 'aaaaaaaaa-xxxxx-xxxx-xxx-bbbbbbbbbb'

# -- Ruta Ficheros JSON ---------------
os.environ['DATAJSON']='/Users/xxxxxxxxxxxxxxxxxxxxxxxxxxxxx/datajson'

In [16]:
import os
import sys
import environ

from requests import Request, Session, get
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from json import load, loads

BADGE = os.getenv('BADGE')
DATAJSON = os.getenv('DATAJSON')
PRO_API_KEY = os.getenv('X-CMC_PRO_API_KEY')

def loadFile_JSON(_path_name):
    try:
        with open(_path_name, 'r') as json_file:
            _lista= load(json_file)
            return _lista
    except:
        sys.exit(f"Error al leer el fichero {_path_name}")


def switch_dict(**params):
    # -- Si command viene mal provoco error --
    assert params['command'] in ['all','id']
    
    return { 
        "all": {"url":"https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest",
                "parameters":{
                    "start":"1",
                    "limit":"5000",
                    "convert":BADGE}},
        "id": {"url":"https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest",
               "parameters":{"id": params['string_id'], "convert": BADGE}},  
    }.get(params['command'], -1)

def getIDfromSymbol(**kwargs):
    """
    Se reciben los simbolos cuyos ID tenemos que encontrar en un fichero json
    """
    in_symbol,in_json = kwargs['symbol'], kwargs['json_data']
    
    _json_file = loadFile_JSON(os.path.join(DATAJSON,in_json))

    lista_id = (lambda l1,l2 : [str(item2['id']) for item1 in l1 for item2 in l2 if item1 == item2['symbol']])(in_symbol.split(','),_json_file['data'])
    
    return f"{','.join(lista_id)}"

# -- OJO que el API devuelve los valores por orden ascendente de ID --
def getCoinData(**kwargs):
    apiurl = switch_dict(command=kwargs['command'], string_id=kwargs['string_id'])
    
    url = apiurl['url']    
    parameters = apiurl['parameters']

    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': PRO_API_KEY
    }

    session = Session()
    session.headers.update(headers)
    
    try:
        response = session.get(url, params=parameters)
        return response
    
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
    


In [2]:
# -- Llamada para obtener el fichero global
# _resp = getCoinData(command='all', string_id="")

_resp = getCoinData(command='id', string_id="1027,2010,1")

if _resp.status_code == 200:
    _resp_json = loads(_resp.text)
else: 
    sys.exit(f"\nEl API devolvío codigo error {_resp.status_code}\n")

In [3]:
for coin in _resp_json['data'].values():
    print(f"id: {coin['id']} - Name: {coin['name']} - \
          Symbol: {coin['symbol']} Price: {coin['quote'][BADGE]['price']}")

id: 1 - Name: Bitcoin -           Symbol: BTC Price: 21503.51187896285
id: 1027 - Name: Ethereum -           Symbol: ETH Price: 1213.4653687268642
id: 2010 - Name: Cardano -           Symbol: ADA Price: 0.4667254216118927


In [15]:
import pandas as pd

_lista = [{'id': coin['id'], 'name': coin['name'], 'symbol': coin['symbol'], \
           'price': coin['quote'][BADGE]['price']} for coin in _resp_json['data'].values()]

pd.DataFrame(_lista,columns = _lista[0].keys()) 

,id,name,symbol,price
0,1,Bitcoin,BTC,21503.511879
1,1027,Ethereum,ETH,1213.465369
2,2010,Cardano,ADA,0.466725


In [21]:
def getIDfromSymbol(**kwargs):
    in_symbol,in_json = kwargs['symbol'], kwargs['json_data']
    
    _json_file = loadFile_JSON(os.path.join(DATAJSON,in_json))

    lista_id = (lambda _symbol_in_string, _json_de_datos : [str(item2['id']) \
                for item1 in _symbol_in_string for item2 in _json_de_datos \
                if item1 == item2['symbol']])(in_symbol.split(','),_json_file['data'])
    
    return f"{','.join(lista_id)}"

In [22]:
getIDfromSymbol(symbol='ETH,BTC,ADA', json_data='completo_reference.json')

'1027,1,2010'